In [75]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [76]:
input=torch.Tensor(1,1,28,28)   # 배치크기(sample 수), 채널, 높이, 너비

In [77]:
conv1 = nn.Conv2d(1, 32, 3, padding=1) # 채널, output_channel, 필터의 수 ,kerne_size(세로3, 가로3)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [78]:
conv2=nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [79]:
pool=nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [80]:
out=conv1(input)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [81]:
out=pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [82]:
out=conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [83]:
out=pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [84]:
out=out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [85]:
fc=nn.Linear(3136, 10)
out=fc(out)
print(out.shape)

torch.Size([1, 10])


In [86]:
print(out)
print(torch.argmax(out))

tensor([[-0.0543,  0.1542, -0.0980,  0.0431,  0.0692,  0.0569, -0.1583,  0.0618,
          0.1331,  0.0583]], grad_fn=<AddmmBackward0>)
tensor(1)


In [87]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [88]:
learning_rate = 0.001
training_epochs = 30
batch_size=100

In [89]:
minst_train=dsets.MNIST(root='data/MINST_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)

minst_test=dsets.MNIST(root='data/MINST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [100]:
fashion_mnist_train=dsets.FashionMNIST(root='data/FashionMINST_data',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
fashion_mnist_test=dsets.FashionMNIST(root='data/FashionMINST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

100%|██████████| 26421880/26421880 [00:17<00:00, 1478756.52it/s]


Extracting data/FashionMINST_data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data/FashionMINST_data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 122385.37it/s]


Extracting data/FashionMINST_data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data/FashionMINST_data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1878880.00it/s]


...

Extracting data/FashionMINST_data\MNIST\raw\train-labels-idx1-ubyte.gz to data/FashionMINST_data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5523627.85it/s]


Extracting data/FashionMINST_data\MNIST\raw\t10k-images-idx3-ubyte.gz to data/FashionMINST_data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data/FashionMINST_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data/FashionMINST_data\MNIST\raw



In [90]:
print(minst_train)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data/MINST_data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [91]:
data_loader=DataLoader(dataset=minst_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [102]:
for X, Y in data_loader:
    print(X.size())
    print(Y.size())
    break

torch.Size([100, 1, 28, 28])
torch.Size([100])


In [92]:
class CNN(nn.Module) :
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2=nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc=nn.Linear(7*7*64, 10, bias=True) #완전연결층
        nn.init.xavier_uniform_(self.fc.weight) #완전연결층 한정 가중치 초기화
    
    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0), -1)
        out=self.fc(out)
        return out    
        

In [93]:
model=CNN().to(device)  # GPU 사용을 위해
criterion = nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [94]:
total_batch=len(data_loader)
print(total_batch)

600


In [95]:
for epoch in range(training_epochs) :
    avg_cost=0
    avg_accuracy=0

    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat=model(X)
        cost = criterion(y_hat, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch
    print(epoch,avg_cost)

0 tensor(0.2256, grad_fn=<AddBackward0>)
1 tensor(0.0630, grad_fn=<AddBackward0>)
2 tensor(0.0463, grad_fn=<AddBackward0>)
3 tensor(0.0374, grad_fn=<AddBackward0>)
4 tensor(0.0313, grad_fn=<AddBackward0>)
5 tensor(0.0260, grad_fn=<AddBackward0>)
6 tensor(0.0217, grad_fn=<AddBackward0>)
7 tensor(0.0180, grad_fn=<AddBackward0>)
8 tensor(0.0162, grad_fn=<AddBackward0>)
9 tensor(0.0132, grad_fn=<AddBackward0>)
10 tensor(0.0097, grad_fn=<AddBackward0>)
11 tensor(0.0098, grad_fn=<AddBackward0>)
12 tensor(0.0086, grad_fn=<AddBackward0>)
13 tensor(0.0070, grad_fn=<AddBackward0>)
14 tensor(0.0071, grad_fn=<AddBackward0>)
15 tensor(0.0054, grad_fn=<AddBackward0>)
16 tensor(0.0041, grad_fn=<AddBackward0>)
17 tensor(0.0048, grad_fn=<AddBackward0>)
18 tensor(0.0050, grad_fn=<AddBackward0>)
19 tensor(0.0036, grad_fn=<AddBackward0>)
20 tensor(0.0029, grad_fn=<AddBackward0>)
21 tensor(0.0039, grad_fn=<AddBackward0>)
22 tensor(0.0043, grad_fn=<AddBackward0>)
23 tensor(0.0031, grad_fn=<AddBackward0>)
24

In [97]:
with torch.no_grad():
    X_test=minst_test.test_data.view(len(minst_test), 1, 28, 28).float().to(device)
    Y_test=minst_test.test_labels.to(device)

    prediction=model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy=correct_prediction.float().mean()
    print(accuracy.item())

c:\Users\admin\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\admin\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


0.9857000112533569
